# MakerSpace Jam

In [ ]:
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Primary Dependencies and Context Setting

In [ ]:
!pip3 install -U -q openai==0.27.8 llama-index==0.8.6 nltk==3.8.1 python-dotenv

### Load the OPENAI API key

In [ ]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]

### Context setting

In [ ]:
from llama_index import ServiceContext
from llama_index.node_parser.simple import SimpleNodeParser
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from llama_index.llms import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding()
chunk_size = 1000
llm = OpenAI(
    temperature=0, 
    model="gpt-3.5-turbo",
    streaming=True
)

service_context = ServiceContext.from_defaults(
    llm=llm, 
    chunk_size=chunk_size,
    embed_model=embed_model
)

text_splitter = TokenTextSplitter(
    chunk_size=chunk_size
)

node_parser = SimpleNodeParser(
    text_splitter=text_splitter
)

## Data Loading

### Wikipedia

In [ ]:
!pip3 install -U -q wikipedia  

### Web

In [ ]:
# list of wikipedia pages to index
webpages = [ "https://www.basketball-reference.com/" ]

##### Simple Web Reader

In [ ]:
# !pip3 install html2text

In [ ]:
# from llama_index import SimpleWebPageReader, TrafilaturaWebReader

# web_docs = SimpleWebPageReader(html_to_text=True).load_data([webpages[0]])
# web_docs[0].text

##### Trafilatura Web Reader (seems better!!)

In [ ]:
!pip3 install trafilatura

In [ ]:
from llama_index import TrafilaturaWebReader

web_docs = TrafilaturaWebReader().load_data([webpages[0]])
web_docs[0].text

## Setup Vector Store

This is intended to be a global vector store to insert the nodes from all data sources

In [ ]:
!pip3 install -U -q chromadb==0.4.6 tiktoken==0.4.0 sentence-transformers==2.2.2 pydantic==1.10.11

In [ ]:
from llama_index import VectorStoreIndex
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
import chromadb

In [ ]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("all_data")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
vector_index = VectorStoreIndex([], storage_context=storage_context, service_context=service_context)

### Construct & Insert Nodes to Store

Parse nodes for each loaded data source and insert it to the vector store.

In [ ]:
# parse nodes from web docs and insert into vector index
for w_doc in web_docs:
    # print(wiki_doc)
    nodes = node_parser.get_nodes_from_documents([w_doc])
    for node in nodes:
        node.metadata = {'title': 'Basketball Stats and History'}
    vector_index.insert_nodes(nodes)

### Setup Metadata Filtering

First, we need to create our `VectoreStoreInfo` object which will hold all the relevant metadata we need for each component (in this case title metadata).

Notice that you need to include it in a text list.

In [ ]:
from llama_index.tools import FunctionTool
from llama_index.vector_stores.types import (
    VectorStoreInfo,
    MetadataInfo,
    ExactMatchFilter,
    MetadataFilters,
)
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine

from typing import List, Tuple, Any
from pydantic import BaseModel, Field

top_k = 3

vector_store_info = VectorStoreInfo(
    content_info="information about NBA players, teams, games, organizations, etc.",
    metadata_info=[MetadataInfo(
        name="title",
        type="str",
        description="info of player, team, organization, games, etc.",
    )]
)

Now we'll create our base PyDantic object that we can use to ensure compatability with our application layer. This verifies that the response from the OpenAI endpoint conforms to this schema.

In [ ]:
class AutoRetrieveModel(BaseModel):
    query: str = Field(..., description="natural language query string")
    filter_key_list: List[str] = Field(
        ..., description="List of metadata filter field names"
    )
    filter_value_list: List[str] = Field(
        ...,
        description=(
            "List of metadata filter field values (corresponding to names specified in filter_key_list)"
        )
    )

Now we can build our function that we will use to query the functional endpoint.

>The `docstring` is important to the functionality of the application.

In [ ]:
def auto_retrieve_fn(
    query: str, filter_key_list: List[str], filter_value_list: List[str]
):
    """Auto retrieval function.

    Performs auto-retrieval from a vector database, and then applies a set of filters.

    """
    query = query or "Query"

    exact_match_filters = [
        ExactMatchFilter(key=k, value=v)
        for k, v in zip(filter_key_list, filter_value_list)
    ]
    retriever = VectorIndexRetriever(
        vector_index, filters=MetadataFilters(filters=exact_match_filters), top_k=top_k
    )
    query_engine = RetrieverQueryEngine.from_args(retriever)

    response = query_engine.query(query)
    return str(response)

Now we need to wrap our system in a tool in order to integrate it into the larger application.

Source Code Here:
- [`FunctionTool`](https://github.com/jerryjliu/llama_index/blob/d24767b0812ac56104497d8f59095eccbe9f2b08/llama_index/tools/function_tool.py#L21)

Extra stuff about fintering:
- [ExactMatchFilter](https://gpt-index.readthedocs.io/en/latest/api_reference/query/retrievers/vector_store.html#llama_index.vector_stores.types.ExactMatchFilter)
- [VectorStoreInfo](https://gpt-index.readthedocs.io/en/latest/api_reference/query/retrievers/vector_store.html#llama_index.vector_stores.types.VectorStoreInfo)

In [ ]:
description = f"""\
Use this tool to look up semantic information about NBA.
The vector database schema is given below:
{vector_store_info.json()}
"""

auto_retrieve_tool = FunctionTool.from_defaults(
    fn=auto_retrieve_fn, 
    name='auto_retrieve_tool',
    description=description, 
    fn_schema=AutoRetrieveModel
)

All that's left to do is attach the tool to an OpenAIAgent and let it rip!

Source Code Here:
- [`OpenAIAgent`](https://github.com/jerryjliu/llama_index/blob/d24767b0812ac56104497d8f59095eccbe9f2b08/llama_index/agent/openai_agent.py#L361)

In [ ]:
from llama_index.agent import OpenAIAgent

agent = OpenAIAgent.from_tools(
    [auto_retrieve_tool], llm=llm, verbose=True
)

In [ ]:
response = agent.chat("Who is the current 'Trending Player' in the NBA?")
print(str(response))

## SQL retriever tool

In [22]:
!pip3 install -q -U sqlalchemy pandas

In [33]:
import pandas as pd

# url_teams = r'https://docs.google.com/spreadsheets/d/1MB1ZsQul4AB262AsaY4fHtGW4HWp2-56zB-E5xTbs2A/edit#gid=972448241'
# teams_df = pd.read_csv(url_teams, on_bad_lines='skip')

teams_df = pd.read_csv('../data/teams.csv')
teams_df.head()